In [58]:
import pandas as pd
import numpy as np
import mca
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
%matplotlib inline
import gc

In [ ]:
from fancyimpute import KNN

In [2]:
df = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/properties_2016.csv')

C:\Users\Stevens\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
target = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/train_2016_v2.csv')

In [4]:
joined_data = pd.merge(target, df, on="parcelid", how="left")

In [ ]:
df2 = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/properties_2017.csv')
target2 = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/train_2017_v2.csv')
joined_data2 = pd.merge(target2, df2, on="parcelid", how="left")

In [5]:
def reduce_mem_usage(props):
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)

    mem_usg = props.memory_usage().sum() / 1024**2 
    return props, NAlist

props, NAlist = reduce_mem_usage(joined_data)

In [59]:
del target
#del target_2017
del df
#del prop_2017

gc.collect()

NameError: name 'target' is not defined

In [6]:
#--- Creating two additional columns each for the month and day ---
joined_data.transactiondate = pd.to_datetime(joined_data.transactiondate,format='%Y-%m-%d')

joined_data['transaction_yr'] = joined_data.transactiondate.dt.weekday.astype(np.int64)
joined_data['transaction_mth'] = joined_data.transactiondate.dt.month.astype(np.int64)
joined_data['transaction_day_of_wk'] = joined_data.transactiondate.dt.weekday.astype(np.int64)

#--- Dropping the 'transactiondate' column now ---
joined_data = joined_data.drop('transactiondate', 1)

In [ ]:
# removing rows with a lot of missing columns
# df = joined_data.ix[joined_data.isnull().sum(axis=1) < 29, ]

In [ ]:
num_cols = ['roomcnt', 'bathroomcnt', 'fullbathcnt', 'bedroomcnt', 
            'fireplacecnt', 'garagecarcnt','numberofstories', 'poolcnt', 
            'unitcnt', 'finishedfloor1squarefeet', 'basementsqft', 'calculatedfinishedsquarefeet', 
            'finishedsquarefeet13', 'finishedsquarefeet6', 'lotsizesquarefeet', 'yardbuildingsqft17', 'yardbuildingsqft26', 
            'poolsizesum', 'finishedsquarefeet15', 'garagetotalsqft', 'landtaxvaluedollarcnt', 
            'structuretaxvaluedollarcnt', 'taxamount', 'transaction_day_of_wk','transaction_mth',
            'transaction_yr', 'taxdelinquencyyear','yearbuilt','latitude','longitude']
num_data = joined_data[num_cols]

In [ ]:
## Bed, bath and room are the only variables with significant amout of outliers.  
outliers = {}
for col in num_data.columns:
    var = num_data[col].dropna()
    index = var[~((var-var.mean()).abs()<2*var.std())]
    outliers[col] = index

outliers = pd.DataFrame.from_dict(outliers).dropna(how='all')


def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    
    return np.where((ys > upper_bound) | (ys < lower_bound))

out_dict = {}
for col in num_data.columns:
    outlier = outliers_iqr(num_data[col])
    out_dict[col] = outlier
    
out_dict = dict((k, out_dict[k]) for k in ('bathroomcnt','bedroomcnt','roomcnt'))

bath = pd.DataFrame(list(list(out_dict['bathroomcnt'])[0])).reset_index()
bed = pd.DataFrame(list(list(out_dict['bedroomcnt'])[0])).reset_index()
room = pd.DataFrame(list(list(out_dict['roomcnt'])[0])).reset_index()

out_bbb = bath.merge(bed, on=0, how='outer').merge(room, on=0, how='outer')
out_bbb.columns = ['index', 'bathroomcnt','bedroomcnt','roomcnt']
out_bbb = out_bbb.iloc[:, 1:4]
# prob not anomalous: joined_data.loc[[14440, 42387, 60385]]

In [ ]:
# only 443 outliers for bedroom - replace with median
bed_ix = (list(list(out_dict['bedroomcnt'])[0]))
bed_median = joined_data.iloc[~joined_data.index.isin(bed_ix)].bedroomcnt.median()
joined_data.loc[(bed_ix), 'bedroomcnt'] = bed_median

In [7]:
#Fill in those properties that have a pool with median pool value
poolsizesum_median = joined_data.loc[joined_data['poolsizesum'] > 0, 'poolsizesum'].median()
joined_data.loc[(joined_data['poolsizesum'].isnull() | 
                joined_data['poolsizesum'] == 0) & 
            ((pd.notnull(joined_data['poolcnt'])) |
            (pd.notnull(joined_data['pooltypeid10'])) |
            (pd.notnull(joined_data['pooltypeid2'])) |
            (pd.notnull(joined_data['pooltypeid7']))), 'poolsizesum'] = poolsizesum_median

In [8]:
#Fill in those properties that have a pool size/type with pool count of 1
joined_data.loc[(joined_data['poolcnt'] != 1) & 
            ((pd.notnull(joined_data['poolsizesum'])) |
            (pd.notnull(joined_data['pooltypeid10'])) |
            (pd.notnull(joined_data['pooltypeid2'])) |
            (pd.notnull(joined_data['pooltypeid7']))), 'poolcnt'] = 1

In [9]:
# fill in fireplace count with mode when there is a fireplace flag is true
joined_data.loc[(joined_data.fireplaceflag == True) & (joined_data.fireplacecnt.isnull()), 'fireplacecnt'] = joined_data.fireplacecnt.dropna().mode()[0]

In [10]:
# number of car garage is more that sqft
garage_median = joined_data[(joined_data.garagetotalsqft != 0)].garagetotalsqft.dropna().median()
joined_data.loc[(joined_data.garagetotalsqft == 0) & (joined_data.garagecarcnt > 0), 'garagetotalsqft'] = garage_median

In [11]:
# unitcnt extreme outliers
unit_median = joined_data.loc[pd.notnull(joined_data['unitcnt'])]['unitcnt'].median()
joined_data.loc[(joined_data['unitcnt'] > 9), 'unitcnt'] = unit_median

In [ ]:
# columns missing 99% of data
missing_df = joined_data.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / joined_data.shape[0]
missing_df.ix[missing_df['missing_ratio']>0.99]

In [66]:
joined_data_sam = joined_data.sample(frac=.05)
train, test = train_test_split(joined_data_sam, test_size=0.2, random_state=4)

In [67]:
num_cols = ['roomcnt', 'bathroomcnt', 'fullbathcnt', 'bedroomcnt', 
            'fireplacecnt', 'garagecarcnt','numberofstories', 'poolcnt', 
            'unitcnt', 'finishedfloor1squarefeet', 'basementsqft', 'calculatedfinishedsquarefeet', 
            'finishedsquarefeet13', 'finishedsquarefeet6', 'lotsizesquarefeet', 'yardbuildingsqft17', 'yardbuildingsqft26', 
            'poolsizesum', 'finishedsquarefeet15', 'garagetotalsqft', 'landtaxvaluedollarcnt', 
            'structuretaxvaluedollarcnt', 'taxamount', 'transaction_day_of_wk','transaction_mth',
            'transaction_yr', 'taxdelinquencyyear','yearbuilt','latitude','longitude', 'logerror']
num_data = train[num_cols]

In [79]:
cat_cols = ['pooltypeid10', 'storytypeid', 'propertylandusetypeid', 'pooltypeid7', 'taxvaluedollarcnt', 
 'buildingqualitytypeid', 'typeconstructiontypeid', 'airconditioningtypeid', 'hashottuborspa',
 'pooltypeid2', 'propertycountylandusecode', 'heatingorsystemtypeid', 'censustractandblock',
 'architecturalstyletypeid']
cat_data = train[cat_cols]

In [29]:
X_train = num_data.drop('logerror', axis=1).values
y_train = num_data['logerror'].values

In [55]:
imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis=0)

In [31]:
X_train = imputer.fit_transform(X_train)

In [32]:
scalar = StandardScaler().fit(X_train)
num_data_norm = scalar.transform(X_train)

In [42]:
estimator = PCA(n_components=.85, svd_solver='full')
num_data_pca = estimator.fit(num_data_norm)

In [ ]:
pca_plot = np.cumsum(np.round(num_data_pca.explained_variance_ratio_, decimals=4)*100)
plt.plot(pca_plot)
print(pca_plot)

In [43]:
X_train = num_data_pca.transform(num_data_norm)

In [47]:
rf = RandomForestRegressor(criterion='mae',n_jobs=-1)

In [48]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [45]:
X_test = test.drop('logerror',axis=1).values
X_test = scalar.transform(X_test)
X_test = num_data_pca.transform(X_test)

In [46]:
y_test = test['logerror'].values

In [49]:
y_preds = rf.predict(X_test)

In [52]:
mean_absolute_error(y_test, y_preds)

0.094385071764189601

In [80]:
cat_data['hashottuborspa'] = cat_data['hashottuborspa'].apply(lambda x: 1 if x == 'True' else 0)

C:\Users\Stevens\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
cat_cols = [
    item for item in cat_cols
                    if item not in ['buildingclasstypeid','fireplaceflag','storytypeid']
]

cat_train = train[cat_cols].copy()
for col in cat_cols:
    if len(train[col].unique()) == 2:
        if cat_train[col].dtype in [np.float32, np.float64,np.int32,np.int64]:
            cat_train[col]=cat_train[col].fillna(0)
        else:
            cat_train[col]=cat_train[col].fillna('no')
    else:
        cat_train[col]=cat_train[col].fillna(cat_train[col].value_counts().index[0])

In [85]:
for col in cat_cols:
    cat_train[col]=cat_train[col].apply(lambda x: str(x))

cat_dummies = pd.get_dummies(cat_train[cat_cols])

In [89]:
cat_dummies

(3611, 6774)

In [90]:
cat_mca = mca.MCA(cat_dummies)